<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/evaluation_v5_precisionatK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install tensorflow==2.8.0 --quiet
!pip install transformers==4.15.0 --quiet
!pip install faiss-gpu transformers torch --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from transformers import BertTokenizer, TFBertModel


In [7]:
from google.colab import drive

# mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from tensorflow.keras.models import load_model
custom_objects = {'TFBertModel': TFBertModel}
model_v5data = load_model('./drive/MyDrive/two_tower_bert_v5.h5', custom_objects=custom_objects)

In [9]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import numpy as np
# cls extraction
def generate_embedding_cls(text, model):
  bert_embedding_model = Model(inputs=[model.input],
                               outputs=model.get_layer('tf.__operators__.getitem').output)
  bert_train_tokenized = bert_tokenizer(text,
                                        max_length=128,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')
  bert_train_inputs = [bert_train_tokenized.input_ids,
                       bert_train_tokenized.token_type_ids,
                       bert_train_tokenized.attention_mask]
  return bert_embedding_model.predict(bert_train_inputs)


def embedding_vectordatabase(embedding):
  embeddings = np.array(embedding, dtype="float32")
  embeddings = embeddings.reshape(-1, embeddings.shape[-1])
  return embeddings

In [11]:
# create vector databases
import faiss
import numpy as np

dimension = 768  # Dimension of BERT-base embeddings
index_cls_v5 = faiss.IndexFlatL2(dimension)

In [12]:
level_four_job = ["Competitive salary for iPhone project management roles",
                  "Relocation benefits for Apple Cupertino campus positions",
                  "Exclusive health and wellness benefits for iPhone development team members",
                  "Stock options and bonuses for iPhone project milestones",
                  "Collaborative workspaces at Apple's Silicon Valley locations",
                  "Professional development programs for iOS project leads",
                  "Confidentiality agreements for upcoming iPhone projects",
                  "Global travel opportunities for iPhone supplier audits",
                  "Flexible working hours for Apple's product development teams",
                  "Team-building retreats for iPhone project managers"]
for input in level_four_job:
  embeddings = generate_embedding_cls(input, model_v5data)
  index_cls_v5.add(embedding_vectordatabase(embeddings))
print(index_cls_v5.ntotal)

10


In [13]:
df_GPT_evaluation =  pd.read_csv("./job_related_GPT_evaluation_updated.csv", error_bad_lines=False)
df_GPT_evaluation

<ipython-input-13-ec4e2fcaa468>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_GPT_evaluation =  pd.read_csv("./job_related_GPT_evaluation_updated.csv", error_bad_lines=False)


,prompt,job_related_info_index,top_3_relevant_blacklist_items
0,"Recently, I've noticed better sleep after expl...",7,"7,2,0"
1,"Recently, I've noticed higher energy levels af...",2,"2,3,0"
2,"Recently, I've noticed less anxiety after taki...",5,"5,0,2"
3,"Recently, I've noticed increased creativity af...",7,"7,0,2"
4,"Recently, I've noticed more patience after par...",2,"2,3,5"
...,...,...,...
295,Since immersing in flexible working hours for ...,8,"8,2,5"
296,Since exploring the advantages of confidential...,6,"6,0,2"
297,Since utilizing confidentiality agreements for...,6,"6,0,2"
298,Since utilizing exclusive health and wellness ...,2,"2,3,5"


In [ ]:
k = 3
indices_ss = []
distance_ss = []
for prompt in df_GPT_evaluation['prompt']:
    embeddings_v5 = generate_embedding_cls(prompt, model_v5data)
    distances_v5, indices_v5 = index_cls_v5.search(embeddings_v5, k)
    indices_ss.append(indices_v5)
    distance_ss.append(distance_ss)


In [ ]:
indices_ss_series = pd.Series(indices_ss)
distance_ss_series = pd.Series(distance_ss)

df_GPT_evaluation['indices_ss'] = indices_ss_series
df_GPT_evaluation['distance_ss'] = distance_ss_series

In [ ]:
df_GPT_evaluation

In [ ]:
df_GPT_evaluation.to_csv('job_related_GPT_evaluation_v5_ss.csv', index=False)

In [ ]:
# based on the difiniate of precision at K and recall at K

# Relevant vs. Recommended
# We have already seen the definition of a relevant items. In the rest of the article we will user relevant and recommended items frequently. Here is a good point to pause and grasp their exact definition.

# # Relevant items are already known in the data set
# Relevant item: Has a True/Actual rating >= 3.5
# Irrelevant item: Has a True/Actual rating < 3.5
# # Recommended items are generated by recommendation algorithm
# Recommended item: has a predicted rating >= 3.5
# Not recommended item: Has a predicted rating < 3.5
# Precision and recall at k: Definition
# Precision at k is the proportion of recommended items in the top-k set that are relevant

# Its interpretation is as follows. Suppose that my precision at 10 in a top-10 recommendation problem is 80%. This means that 80% of the recommendation I make are relevant to the user.

# Mathematically precision@k is defined as follows:

# Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
# Recall at k is the proportion of relevant items found in the top-k recommendations

# Suppose that we computed recall at 10 and found it is 40% in our top-10 recommendation system. This means that 40% of the total number of the relevant items appear in the top-k results.

# Mathematically recall@k is defined as follows:

# Recall@k = (# of recommended items @k that are relevant) / (total # of relevant items)

# and the blacklist

# [
#     "Competitive salary for iPhone project management roles",
#     "Relocation benefits for Apple Cupertino campus positions",
#     "Exclusive health and wellness benefits for iPhone development team members",
#     "Stock options and bonuses for iPhone project milestones",
#     "Collaborative workspaces at Apple's Silicon Valley locations",
#     "Professional development programs for iOS project leads",
#     "Confidentiality agreements for upcoming iPhone projects",
#     "Global travel opportunities for iPhone supplier audits",
#     "Flexible working hours for Apple's product development teams",
#     "Team-building retreats for iPhone project managers"
# ]

# by given csv file, take the column "top_3_relevant_blacklist_items" as relavent items and calculate the precision at K and recall at K of indices_ss and save these two as two columns in the csv file.